In [1]:
class LAADS_FILE:
  
  @classmethod
  def generate(cls,file):

    from datetime import datetime

    cls.file = file
    cls.date = datetime.strptime(cls.file.split('.')[1].split('T')[0],'%Y-%m-%d')

    return cls


In [13]:
import os

# directorio
dir = './LAADS_querys/'

# archivos
files = os.listdir(dir)

# lista de achvos
files_list = list(map( LAADS_FILE.generate, files))

# fecha actualizacion
max_date = max( list(map(lambda x: x.date,files_list)) )

# archivo actual
file_actualizado = dir + list(filter(lambda x: x.date == max_date, files_list))[0].file
file_actualizado

'./LAADS_querys/LAADS_query.2022-07-25T21_45.json'